# Vicon CSV → Quantity of Motion (QdM)

This notebook computes:
- 3D marker trajectories from Vicon CSV exports
- Quantity of Motion (QdM) for wrists and head
- Exports results to an Excel summary file

Author: Matys Précloux  
Project: SYNCOGEST